In [2]:
import numpy as np
from tqdm import *
from astropy.io import fits
from skimage import exposure

Define simualtion data names

In [2]:
# super-alfvénic
sup_alfven_1 = 'dens.b1_p01_num1.fits'
sup_alfven_2 = 'dens.b1_p01_num2.fits'
# sub-alfvénic
sub_alfven_1 = 'dens.b01_p01_num1.fits'
sub_alfven_2 = 'dens.b01_p01_num2.fits'

In [3]:
# code to extract the images
def makeimages(data, imsize, nx, sh, nsh, filename, killpwr=False):
    imsize = int(imsize)
    dsh = data.shape
    newdata = data
    cube = np.zeros([imsize, imsize, int(dsh[2]*nsh*nsh*nx*nx)])
    iterct = 0
    for s in np.arange(dsh[2]):
        if killpwr:
            img = data[:, :, s]
            fftsl= np.fft.fft2(img)
            normalized = fftsl/np.abs(fftsl)
            newimg = np.real(np.fft.ifft2(normalized))
            newdata[:, :, s] = newimg
    for s in tqdm(np.arange(dsh[2])):
        for r in np.arange(nsh):
            for q in np.arange(nsh):
                fullimg = np.roll(np.roll(newdata[:, :, s], int(r*sh+s), axis=0), int(q*sh+s), axis=1)
                for i in np.arange(nx):
                    for j in np.arange(nx):
                        toimg = fullimg[int(i*imsize):int((i+1)*imsize), int(j*imsize):int((j+1)*imsize)]
                        exeq = exposure.equalize_hist(toimg)
                        cube[:, :, iterct] = exeq
                        iterct = iterct +1
                        #plt.imshow(toimg)
                        #print(np.max(toimg))
                        exeq = exeq.reshape([imsize, imsize, 1])*np.ones([imsize, imsize, 3])*255
                        img = Image.fromarray(exeq.astype('int8'), mode='RGB')
    return cube

Some basic parameters for image extraction

In [ ]:
# The size of the images to make
imsize = 128
# The "striding"
nsh = 2
# The stride size
sh = imsize/nsh
# The number of images across a cube
nx = 4

Extract the images for the sub-Alfvénic simulation

In [18]:
#Make the training data for sub-alfvenic
cube = fits.getdata(sub_alfven_1)
data = np.log10(cube)
data = (data- np.nanmin(data))/(np.nanmax(data)- np.nanmin(data))*255.0
shcube = data.shape
data[~np.isfinite(data)] = 0.

train_x = makeimages(data[:, :, 0:96*4], int(imsize), nx, sh, nsh, killpwr=False)
np.save('b1p.01XY_train_ss1_sh_64_ims_128.npy', train_x)

#Make the test data for sub-alfvenic
test_x = makeimages(data[:, :, (128-16-8)*4:-8*4], int(imsize), nx, sh, nsh, killpwr=False)
np.save('b1p.01XY_test_ss1_sh_64_ims_128.npy', test_x)

#repeat, but for FFP tests 

train_x = makeimages(data[:, :, 0:96*4], int(imsize), nx, sh, nsh, killpwr=True)
np.save('b1p.01XY_train_kp_ss1_sh_64_ims_128.npy', train_x)

#Make the test data for sub-alfvenic
test_x = makeimages(data[:, :, (128-16-8)*4:-8*4], int(imsize), nx, sh, nsh, killpwr=True)
np.save('b1p.01XY_test_kp_ss1_sh_64_ims_128.npy', test_x)

#now repeat for the second snapshot

cube = fits.getdata(sub_alfven_2)
data = np.log10(cube)
data = (data- np.nanmin(data))/(np.nanmax(data)- np.nanmin(data))*255.0
shcube = data.shape
data[~np.isfinite(data)] = 0.

train_x = makeimages(data[:, :, 0:96*4], int(imsize), nx, sh, nsh, killpwr=False)
np.save('b1p.01XY_train_ss1_sh_64_ims_128_2.npy', train_x)

#Make the test data for sub-alfvenic
test_x = makeimages(data[:, :, (128-16-8)*4:-8*4], int(imsize), nx, sh, nsh, killpwr=False)
np.save('b1p.01XY_test_ss1_sh_64_ims_128_2.npy', test_x)

#repeat, but for FFP tests 

train_x = makeimages(data[:, :, 0:96*4], int(imsize), nx, sh, nsh, killpwr=True)
np.save('b1p.01XY_train_kp_ss1_sh_64_ims_128_2.npy', train_x)

#Make the test data for sub-alfvenic
test_x = makeimages(data[:, :, (128-16-8)*4:-8*4], int(imsize), nx, sh, nsh, killpwr=True)
np.save('b1p.01XY_test_kp_ss1_sh_64_ims_128_2.npy', test_x)


Extract the images for the super-Alfvénic simulation

In [ ]:
#Make the training data for super-alfvenic
cube = fits.getdata(super_alfven_1)
data = np.log10(cube)
data = (data- np.nanmin(data))/(np.nanmax(data)- np.nanmin(data))*255.0
shcube = data.shape
data[~np.isfinite(data)] = 0.

train_x = makeimages(data[:, :, 0:96*4], int(imsize), nx, sh, nsh, killpwr=False)
np.save('b.1p.01XY_train_ss1_sh_64_ims_128.npy', train_x)

#Make the test data for super-alfvenic
test_x = makeimages(data[:, :, (128-16-8)*4:-8*4], int(imsize), nx, sh, nsh, killpwr=False)
np.save('b.1p.01XY_test_ss1_sh_64_ims_128.npy', test_x)

#repeat, but for FFP tests 

train_x = makeimages(data[:, :, 0:96*4], int(imsize), nx, sh, nsh, killpwr=True)
np.save('b.1p.01XY_train_kp_ss1_sh_64_ims_128.npy', train_x)

#Make the test data for super-alfvenic
test_x = makeimages(data[:, :, (128-16-8)*4:-8*4], int(imsize), nx, sh, nsh, killpwr=True)
np.save('b.1p.01XY_test_kp_ss1_sh_64_ims_128.npy', test_x)

#now repeat for the second snapshot

cube = fits.getdata(super_alfven_2)
data = np.log10(cube)
data = (data- np.nanmin(data))/(np.nanmax(data)- np.nanmin(data))*255.0
shcube = data.shape
data[~np.isfinite(data)] = 0.

train_x = makeimages(data[:, :, 0:96*4], int(imsize), nx, sh, nsh, killpwr=False)
np.save('b.1p.01XY_train_ss1_sh_64_ims_128_2.npy', train_x)

#Make the test data for super-alfvenic
test_x = makeimages(data[:, :, (128-16-8)*4:-8*4], int(imsize), nx, sh, nsh, killpwr=False)
np.save('b.1p.01XY_test_ss1_sh_64_ims_128_2.npy', test_x)

#repeat, but for FFP tests 

train_x = makeimages(data[:, :, 0:96*4], int(imsize), nx, sh, nsh, killpwr=True)
np.save('b.1p.01XY_train_kp_ss1_sh_64_ims_128_2.npy', train_x)

#Make the test data for super-alfvenic
test_x = makeimages(data[:, :, (128-16-8)*4:-8*4], int(imsize), nx, sh, nsh, killpwr=True)
np.save('b.1p.01XY_test_kp_ss1_sh_64_ims_128_2.npy', test_x)
